This is a tiny amount of scaffolding around the directed homophilic preferential attachment code written by Lisette Espin and others ([available here](https://github.com/gesiscss/Homophilic_Directed_ScaleFree_Networks)). In particular, we replicate a simple case where homophily within groups can lead to inequities between groups, defined as underrepresentation of minorities in the top k% of ranks.

In [82]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import time

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

from paper.DPAH import DPAH
from DPAH2 import DPAH2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
def compute_inequity_old(g, k):
    """Compute the proportion of minorities in the top k ranks of g"""
    node_pageranks = nx.pagerank(g)
    node_pageranks_sorted = sorted(node_pageranks.items(), key=lambda x: x[1], reverse=True)
    top_k = node_pageranks_sorted[:k]
    
    num_top_k_minority = sum([g.nodes[node_id]['m'] for (node_id, _) in top_k])
    
    return num_top_k_minority / k    

In [68]:
def compute_inequity(g, k):
    """Compute the proportion of allies and minorities in the top k ranks of g"""
    node_pageranks = nx.pagerank(g)
    node_pageranks_sorted = sorted(node_pageranks.items(), key=lambda x: x[1], reverse=True)
    top_k = node_pageranks_sorted[:k]
    
    num_top_k_allies = 0
    num_top_k_minority = 0
    
    for (node_id, _) in top_k:
        if g.nodes[node_id]['m'] == 1: # is an ally:
            num_top_k_allies += 1
        elif g.nodes[node_id]['m'] == 2: # is minority
            num_top_k_minority += 1
    
    return num_top_k_allies / k, num_top_k_minority / k

In [80]:
# Number of nodes
N = 1000

# Top beta% of rankings to consider for inequity
beta = 0.05

# Top k nodes to consider for inequity. k = N * beta
k = int(N * beta)

# Fraction minority
fm = 0.2

# Fraction allies
fa = 0

# homophily within majority group
h_MM = 0.8

# homophily within minority group
h_mm = 0.8

print(k)

50


In [83]:
start = time.time()

results = []

for i in range(3):
    g = DPAH(N=N, fm=fm, d=0.0015, plo_M=3, plo_m=3, h_MM=h_MM, h_mm=h_mm, verbose=False, seed=i)
    prop_top_k_minority = compute_inequity_old(g, k)
    results.append(prop_top_k_minority)
    
end = time.time()

end - start

15.290210008621216

In [84]:
results

[0.06, 0.14, 0.08]

With strong homophily (h_MM=h_mm=0.8), in all five runs minorities are underrepresented in the top 5% (50) of ranks. In particular, with equity, we'd expect an average representation of 20% in the top ranks.

In [ ]:
results = []

for fraction_allies in np.linspace(0, 1 - fm, 20):
    print(fraction_allies)
    for i in range(3):
        g = DPAH2(
            N=N,
            fm=fm,
            fa=fraction_allies, # Fraction that are allies
            d=0.0015,
            plo_M=3,
            plo_m=3,
            h_MM=h_MM,
            h_mm=h_mm,
            verbose=False,
            seed=i)
        prop_top_k_ally, prop_top_k_minority = compute_inequity(g, k)
        results.append((fraction_allies, prop_top_k_ally, prop_top_k_minority))

0.0


In [89]:
results

[(0.0, 0.0, 0.06),
 (0.0, 0.0, 0.14),
 (0.0, 0.0, 0.08),
 (0.8, 0.34, 0.66),
 (0.8, 0.24, 0.76),
 (0.8, 0.3, 0.7)]